In [2]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import cv2
import locale
import os

### Load in nnhe array Data

In [47]:
meta = []

with open('data-he.csv') as f:
    for line in f.readlines():
        meta.append(eval(line))

meta = np.array(meta)
np.save('nnhe-meta-simple-minturn-10-4', meta)

### Load in player hand data

In [48]:
player_hands = []

with open('data-ph.csv') as f:
    for line in f.readlines():
        player_hands.append(eval(line))

player_hands = np.array([arr.flatten() for arr in np.array(player_hands)])
np.save('nnhe-player-hands-simple-minturn-10-4', player_hands)

### From np file

In [49]:
meta = np.load('nnhe-meta-simple-minturn-10-4.npy')
player_hands = np.load('nnhe-player-hands-simple-minturn-10-4.npy')

### split train and test data

In [50]:
i = int(len(meta) / 2)
j = int(len(meta) * 3 / 4)

trainX = meta[:i]
testX = meta[i:j]
endX = meta[j:]

trainY = player_hands[:i]
testY = player_hands[i:j]
endY = player_hands[j:]

### Create model

In [51]:
model = Sequential()
model.add(Dense(100, input_dim=trainX.shape[1], activation="relu"))
model.add(Dense(85, activation="sigmoid"))
model.add(Dense(60, activation="sigmoid"))
model.add(Dense(52, activation="sigmoid"))

opt = Adam(
       learning_rate = 0.001,     # 0.001
       beta_1=0.9,                # 0.9
       beta_2=0.999,              # 0.999
       epsilon=0.0001,            # 1e-7
       amsgrad=False,             # False
       name="Adam"
      )

model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['accuracy', 'mean_squared_error', 'categorical_crossentropy'])

### Fit

In [55]:
model.fit(
        trainX, trainY,
        validation_data=(testX, testY),
        epochs=2000, batch_size=50
)

Epoch 1/2000
9677/9677 [==============================] - 10s 1ms/step - loss: 0.1010 - accuracy: 0.0466 - mean_squared_error: 0.1010 - categorical_crossentropy: 32.1184 - val_loss: 0.1039 - val_accuracy: 0.0311 - val_mean_squared_error: 0.1039 - val_categorical_crossentropy: 32.3734
Epoch 2/2000
9677/9677 [==============================] - 10s 1ms/step - loss: 0.1009 - accuracy: 0.0461 - mean_squared_error: 0.1009 - categorical_crossentropy: 32.1164 - val_loss: 0.1040 - val_accuracy: 0.0518 - val_mean_squared_error: 0.1040 - val_categorical_crossentropy: 32.3688
Epoch 3/2000
9677/9677 [==============================] - 10s 1ms/step - loss: 0.1009 - accuracy: 0.0465 - mean_squared_error: 0.1009 - categorical_crossentropy: 32.1156 - val_loss: 0.1038 - val_accuracy: 0.0549 - val_mean_squared_error: 0.1038 - val_categorical_crossentropy: 32.3508
Epoch 4/2000
9677/9677 [==============================] - 10s 1ms/step - loss: 0.1009 - accuracy: 0.0465 - mean_squared_error: 0.1009 - categoric

In [53]:
model.fit(
        trainX, trainY,
        validation_data=(testX, testY),
        epochs=50, batch_size=50
)

Epoch 1/50
9677/9677 [==============================] - 11s 1ms/step - loss: 0.1014 - accuracy: 0.0457 - mean_squared_error: 0.1014 - categorical_crossentropy: 32.1905 - val_loss: 0.1044 - val_accuracy: 0.0424 - val_mean_squared_error: 0.1044 - val_categorical_crossentropy: 32.4756or: 0.
Epoch 2/50
9677/9677 [==============================] - 11s 1ms/step - loss: 0.1014 - accuracy: 0.0451 - mean_squared_error: 0.1014 - categorical_crossentropy: 32.1877 - val_loss: 0.1044 - val_accuracy: 0.0406 - val_mean_squared_error: 0.1044 - val_categorical_crossentropy: 32.4760
Epoch 3/50
9677/9677 [==============================] - 11s 1ms/step - loss: 0.1014 - accuracy: 0.0455 - mean_squared_error: 0.1014 - categorical_crossentropy: 32.1860 - val_loss: 0.1043 - val_accuracy: 0.0394 - val_mean_squared_error: 0.1043 - val_categorical_crossentropy: 32.4527
Epoch 4/50
9677/9677 [==============================] - 10s 1ms/step - loss: 0.1014 - accuracy: 0.0451 - mean_squared_error: 0.1014 - categorical

In [54]:
model.save('nnhe-simple-minturn-10-4.h5')

# End - Testing

In [30]:
def largest(arr, n):
    new_arr = [*arr]
    for i in range(n):
        new_arr[new_arr.index(max(new_arr))] = -1
    return [1 if new_arr[i] == -1 else 0 for i in range(52)]

In [31]:
t = 6

X = endX
Y = endY
P = model.predict(np.array([X[t]]))[0]


np.set_printoptions(suppress=True)

print("test case:", t)
# print("stage:", sum(list(X[t][i]).count(-1) + list(X[t][i]).count(1) for i in range(4)))
print()
print("meta:")
print(X[t])
print()
print("Prediction:")
print(P)
print()
print("  pred:"," ".join('{:.2f}'.format(round(e,2)) for e in P))
print("actual:", " ".join(f"{e}   "[:4] for e in Y[t]))
print("\n", "-" * 150, "\n")
print("rounded:", *[int(round(p)) for p in P])
print(" actual:", *[e for e in Y[t]])
print("largest:", *largest(P, 15))
print()
print("mean squared diff:", sum((P - Y[t]) ** 2) / 52)

test case: 6

meta:
[ 0.44764248  0.03002224  0.          0.          0.39111662  0.
  0.04063429  0.          0.          0.00194905  0.          0.00230315
  0.          0.00525209  0.          0.3363671   0.          0.45838246
  0.01972279  0.          0.95064229  0.          0.00408504  0.00584477
  0.01321854  1.          0.01950026  0.43526301  0.02117212  0.22373372
  0.30191153  0.          0.08411534  0.99429226  0.90091896  0.
  0.          0.          0.00351636  0.49991715  0.16778597  0.23186406
  0.26220354  0.55138099  0.          0.45020029  0.95724416  0.
  0.00234763  0.          0.00109335  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.  